# Interpretability of the Image Quality Metrics (IQMs) of MRIQC

MRIQC is a powerful tool to assess the quality of MR images in a research study. In addition to a visual report, a number of image quality metrics (IQMs) is generated. However, there is a large number of these metrics and it is not immediately obvious which IQM a researcher should pay most attention to when deciding over the quality of a given image.

In this notebook, we will explore these issues in the MR-ART dataset, to provide researchers guidance in interpreting the IQMs from MRIQC.

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

First, let's import the data. The [Movement-Related Artifacts (MR-ART)](https://www.nature.com/articles/s41597-022-01694-8) dataset contains the T1-weighted images of 148 healthy subjects. Each subject has been acquired under three motion conditions:

1. no head movement
2. little head movement
3. much head movement

The motion was artifically induced by giving the subjects cues when to node their head.

The images were given to two expert raters, who rated the images in their quality, with ratings

1. good quality
2. medium quality
3. bad quality.

What we are interested in here are the IQMs and the ratings scores.

In [4]:
# set path to the data, adjust to where you saved yours
path_data = os.path.abspath("../../data/")
# import IQMs
iqms = pd.read_csv(os.path.join(path_data, "group_T1w.tsv"), sep="\t")
# import rating scores
scores = pd.read_csv(os.path.join(path_data, "scores.tsv"), sep="\t")

In [ ]:
def add_condition_column(scores):
    stand = scores["bids_name"].str.contains("standard")
    hm1 = scores["bids_name"].str.contains("motion1")
    hm2 = scores["bids_name"].str.contains("motion2")
    conditions = [
        (stand == True),
        (hm1 == True),
        (hm2 == True)]
    choices = [1, 2, 3]
    scores['condition'] = np.select(conditions, choices)
    print(scores)